In [ ]:
import pandas as pd
import glob
from tqdm import tqdm
import gc
from collections import defaultdict
import datetime
import json
import itertools
import matplotlib.pyplot as plt
import copy
import math
from collections import Counter
import numpy as np
import gc
import time
import random
import pickle
import os
import shutil

import whoosh_utils

In [ ]:
import cupy as cp

In [ ]:
"""
a1 = np.arange(100000)
a2 = np.arange(100000)
for _ in tqdm(range(100000)):
    d = np.intersect1d(a1, a2)
    
a1 = cp.arange(100000)
a2 = cp.arange(100000)
for _ in tqdm(range(100000)):
    d = cp.intersect1d(a1, a2)
"""

In [ ]:

# TRAIN_PKL_PATH = '/kaggle/input/sa-cpc-title-abst-clm-desc-10-5-use-allpub-svpkl/'
# TRAIN_PKL_PATH = ''

# TRAIN_PKL_PATH = '/kaggle/input/sa-cpc-title-abst-clm-inf-desc-10-5-save-pkl/'

TRAIN_PKL_PATH = '/kaggle/input/sa-cpc-title-abst-clm-inf-cupy-save/'

In [ ]:
IS_TRAIN = False
IS_REDUCE_TRAIN = False

IS_ADD_NEG_0_PATTERN = True

In [ ]:
NEG_MAX_COUNT = 1000
PUB_MAX_COUNT = 50 + NEG_MAX_COUNT # 10
SUB_MAX_COUNT = 50 + NEG_MAX_COUNT # 100

PATTERN_NUM_MAX = 10000

CHAR_LIMIT = 9000

In [ ]:
NEG_WEIGHT = 0.01 # 0.1

In [ ]:
T0 = 1
T1 = 0.1

max_time = 1 # 5

In [ ]:
if IS_TRAIN:
    nn_path = '/kaggle/input/create-valid-index-add-claim/nn_df_for_index.csv'
    nn_df = pd.read_csv(nn_path)
    print('len(nn_df)', len(nn_df))
    
    nn_df.to_csv('nn_df_for_index.csv', index=False)
else:
    nn_df = pd.read_csv('/kaggle/input/uspto-explainable-ai/test.csv')
    nn_df.to_csv('nn_df_for_index.csv', index=False)

In [ ]:
import os
os.makedirs('nn_nn', exist_ok=True)
os.makedirs('reduce', exist_ok=True)

In [ ]:
%%writefile nn_nn/publication_to_word.py

import pandas as pd
import glob
from tqdm import tqdm
import gc
from collections import defaultdict
import datetime
import json
import itertools
import matplotlib.pyplot as plt
import copy
import math
from collections import Counter
import numpy as np
import gc
import time
import random
import pickle
import os

nn_df = pd.read_csv('nn_df_for_index.csv')

use_pub = set()
for nn in nn_df.values:
    for n in nn:
        use_pub.add(n)

print(len(use_pub))

# publication_to_word = pickle.load(open(base_path + 'publication_to_word.pkl', 'rb'))

publication_to_word = {}
for i in tqdm(range(21)):
    _publication_to_word = pickle.load(open(f'/kaggle/input/cpc-title-abst-claim-inf-chunk/publication_to_word_{i}.pkl', 'rb'))
    for pub, word in _publication_to_word.items():
        if pub in use_pub:
            publication_to_word[pub] = word
    
print('len(publication_to_word)', len(publication_to_word))

pickle.dump(publication_to_word, open(f'reduce/publication_to_word.pkl', 'wb'))

In [ ]:
if IS_TRAIN and TRAIN_PKL_PATH != '':
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/publication_to_word.pkl', 'reduce/publication_to_word.pkl')
else:
    !python nn_nn/publication_to_word.py

# publication to int  


In [ ]:
%%writefile nn_nn/pub_to_num.py

import pandas as pd
import glob
from tqdm import tqdm
import gc
from collections import defaultdict
import datetime
import json
import itertools
import matplotlib.pyplot as plt
import copy
import math
from collections import Counter
import numpy as np
import gc
import time
import random
import pickle
import os

patent_metadata_df = pd.read_parquet('/kaggle/input/uspto-explainable-ai/patent_metadata.parquet',
                                     columns=['publication_number'])

pub_to_num = dict()
num_to_pub = dict()
for num, pub in enumerate(patent_metadata_df['publication_number'].values):
    pub_to_num[pub] = num
    num_to_pub[num] = pub

pickle.dump(pub_to_num, open(f'reduce/pub_to_num.pkl', 'wb'))
pickle.dump(num_to_pub, open(f'reduce/num_to_pub.pkl', 'wb'))

In [ ]:
if IS_TRAIN and TRAIN_PKL_PATH != '':
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/pub_to_num.pkl', 'reduce/pub_to_num.pkl')
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/num_to_pub.pkl', 'reduce/num_to_pub.pkl')
else:
    !python nn_nn/pub_to_num.py

# word to publication、nn_df以外のwordは消す  


In [ ]:
os.makedirs('word_to_pub_set_chunk', exist_ok=True)

In [ ]:
%%writefile nn_nn/reduce_pkl.py

import pandas as pd
import glob
from tqdm import tqdm
import gc
from collections import defaultdict
import datetime
import json
import itertools
import matplotlib.pyplot as plt
import copy
import math
from collections import Counter
import numpy as np
import gc
import time
import random
import pickle
import os

base_path = 'reduce/'
pub_to_num = pickle.load(open(base_path + 'pub_to_num.pkl', 'rb'))
num_to_pub = pickle.load(open(base_path + 'num_to_pub.pkl', 'rb'))

nn_df = pd.read_csv('nn_df_for_index.csv')

use_pub = set()
for nn in nn_df.values:
    for n in nn:
        use_pub.add(pub_to_num[n])

print(len(use_pub))

publication_to_word = pickle.load(open('reduce/' + 'publication_to_word.pkl', 'rb'))
publication_to_word_new = defaultdict(list)
for pub, word in tqdm(publication_to_word.items()):
    publication_to_word_new[pub_to_num[pub]] = word
publication_to_word = publication_to_word_new


base_path = '/kaggle/input/cpc-title-abst-claim-inf/'
word_to_pubcount = pickle.load(open(base_path + 'word_to_pubcount.pkl', 'rb'))

use_words = set()
for pub in tqdm(use_pub):
    words = publication_to_word[pub]
    use_words.update(words)
print('len(use_words)', len(use_words))


for i in tqdm(range(21)):
    print(f'chunk_{i}')
    word_to_pub_set = {}
    _word_to_pub_set = pickle.load(open(f'/kaggle/input/cpc-title-abst-claim-inf-chunk/word_to_pub_set_{i}.pkl', 'rb'))
    for word, pub_set in _word_to_pub_set.items():
        if word in use_words:            
            pub_set = np.array([pub_to_num[pub] for pub in pub_set], dtype='int32')
            word_to_pub_set[word] = pub_set

    pickle.dump(word_to_pub_set, open(f'reduce/word_to_pub_set_chunk_{i}.pkl', 'wb'))
    
    del word_to_pub_set, _word_to_pub_set
    gc.collect()

gc.collect()

# wordを数値に変換 (省メモリ化のため)
word_to_number = pickle.load(open(base_path + 'word_to_number.pkl', 'rb'))
# 数値をwordに変換 例: number_to_word[ti:device] → 10
number_to_word = pickle.load(open(base_path + 'number_to_word.pkl', 'rb'))

remove_words = [w for w in number_to_word.keys() if w not in use_words]
print('len(remove_words)', len(remove_words))
for number in remove_words:
    word = number_to_word[number]
    word_to_number.pop(word)
    number_to_word.pop(number)

print('len(word_to_number)', len(word_to_number))
print('len(number_to_word)', len(number_to_word))


remove_words = [w for w in word_to_pubcount.keys() if w not in use_words]
print('len(remove_words)', len(remove_words))
for word in remove_words:
    word_to_pubcount.pop(word)
print('len(word_to_pubcount)', len(word_to_pubcount))

pickle.dump(word_to_number, open(f'reduce/word_to_number.pkl', 'wb'))
pickle.dump(number_to_word, open(f'reduce/number_to_word.pkl', 'wb'))
pickle.dump(word_to_pubcount, open(f'reduce/word_to_pubcount.pkl', 'wb'))
pickle.dump(publication_to_word, open(f'reduce/publication_to_word.pkl', 'wb'))

In [ ]:
if IS_TRAIN and TRAIN_PKL_PATH != '':
    # shutil.copy(f'{TRAIN_PKL_PATH}reduce/word_to_pub_set.pkl', 'reduce/word_to_pub_set.pkl')
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/word_to_number.pkl', 'reduce/word_to_number.pkl')
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/number_to_word.pkl', 'reduce/number_to_word.pkl')
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/word_to_pubcount.pkl', 'reduce/word_to_pubcount.pkl')
    shutil.copy(f'{TRAIN_PKL_PATH}reduce/publication_to_word.pkl', 'reduce/publication_to_word.pkl')
else:
    !python nn_nn/reduce_pkl.py

# load index 

In [ ]:
if IS_TRAIN:
    # train_idx = whoosh_utils.load_index('/kaggle/input/create-valid-index-add-claim/test_index')
    train_idx = whoosh_utils.load_index('/kaggle/input/uspto-test-index/test_index')
    searcher = whoosh_utils.get_searcher(train_idx)
    qp = whoosh_utils.get_query_parser()

    # query = 'ti:balloons OR ti:string'
    query = '(cpc:A01H6/42 ab:cuttings) OR (ab:regal-ab:rooting) OR (ab:magenta-ab:umbels) OR (ti:geranium-ti:variety-ab:cherry-ab:foliage-ab:pink-clm:geranium) OR (ti:bravo-ab:bicolored) OR (ab:foliage-ab:geranium-ab:regal-clm:geranium-detd:duchess) OR (ti:debutante-ti:pelargonium) OR (ti:geranium-ab:bright-ab:garden-ab:geranium-ab:mounded-ab:zonal-clm:geranium-clm:oglger14007-clm:zonal-detd:oglger14007) OR (ti:geranium-ti:variety-ab:blossoms-ab:foliage-ab:orchid-clm:geranium) OR (ti:louise-ab:ivy) OR (ti:hots) OR (ti:geranium-ti:variety-ab:orange-ab:rain-clm:geranium)'
    print(whoosh_utils.execute_query(query, qp, searcher)[:5])

In [ ]:
if IS_TRAIN and TRAIN_PKL_PATH != '':
    chunk_path = TRAIN_PKL_PATH + 'reduce/'
else:
    chunk_path = 'reduce/'
word_to_pub_set = {}
for i in tqdm(range(21)):
    _word_to_pub_set = pickle.load(open(chunk_path + f'word_to_pub_set_chunk_{i}.pkl', 'rb'))
    for k, v in _word_to_pub_set.items():
        # word_to_pub_set[k] = v
        word_to_pub_set[k] = cp.array(np.sort(v), dtype=cp.int32)

In [ ]:
%%time
import pickle


base_path = 'reduce/'

# wordを数値に変換 (省メモリ化のため)
word_to_number = pickle.load(open(base_path + 'word_to_number.pkl', 'rb'))

# 数値をwordに変換 例: number_to_word[ti:device] → 10
number_to_word = pickle.load(open(base_path + 'number_to_word.pkl', 'rb'))

In [ ]:
word_to_pubcount = pickle.load(open(base_path + 'word_to_pubcount.pkl', 'rb'))

In [ ]:
"""
特定のwordを持つpublication_numberの集合 word(cpc,title,abstract)
recallなどを高速に計算するために、集合で保持
例: word_to_pub_set[cpc1] → set([pub1, pub2, pub3])
"""

# word_to_pub_set = pickle.load(open(base_path + 'word_to_pub_set.pkl', 'rb'))

In [ ]:
publication_to_word = pickle.load(open(base_path + 'publication_to_word.pkl', 'rb'))

In [ ]:
pub_to_num = pickle.load(open(base_path + 'pub_to_num.pkl', 'rb'))
num_to_pub = pickle.load(open(base_path + 'num_to_pub.pkl', 'rb'))

In [ ]:
counts = []
for _, cnt in word_to_pubcount.items():
    counts.append(cnt)
plt.hist(counts)

In [ ]:
nn_df = pd.read_csv('nn_df_for_index.csv')

In [ ]:
for i in range(nn_df.values.shape[0]):
    for j in range(nn_df.values.shape[1]):
        nn_df.values[i, j] = pub_to_num[nn_df.values[i, j]]

In [ ]:
neighbors = nn_df.values[:, 1:]

In [ ]:

word_pattern_to_pub_set_list = [defaultdict(set) for _ in range(len(neighbors))]
word_pattern_to_pub_set_all = dict()
add_count_list = []


for i in tqdm(range(len(neighbors))):

    word_pattern_to_pub_set = word_pattern_to_pub_set_list[i]
    add_list = []
    

    nn_list = neighbors[i]
    # nn_array = np.array(neighbors[i], dtype=np.int32)
    
    nn_array = neighbors[i]
    nn_array = np.sort(nn_array)
    nn_array = cp.array(nn_array, dtype=np.int32)
    nn_set = set(nn_list)

    # use_words_seen = set()
    use_words_len_max = -1

    # 行ごとにcacheをリセットして、メモリが増えすぎないようにする
    pub_set_cache = {}
    use_words_cache = {}
    
    t_sum = 0
    for r in range(1, 3):
        debug_count=0
        for nn_comb in itertools.combinations(nn_list, r):
            counts = []
            if nn_comb[0] not in use_words_cache:
                use_words_cache[nn_comb[0]] = set([w for w in publication_to_word[nn_comb[0]]])
            use_words = use_words_cache[nn_comb[0]].copy()
            for nn in nn_comb[1:]:
                if not nn in use_words_cache:
                    use_words_cache[nn] = set([w for w in publication_to_word[nn]])
                use_words &= use_words_cache[nn]
                
            use_words = tuple(sorted(list(use_words)))

            if len(use_words) == 0: #  or use_words in use_words_seen
                continue

            use_words_sorted = sorted(use_words, key=lambda x: word_to_pubcount[x])
            
            use_words = tuple([w for w in use_words if w in set(use_words_sorted)])
            # use_words_len_max = max(len(use_words), use_words_len_max)
            
            use_words_final = []
            """
            # 要素数1万以上の集合は、setを作る時間の短縮のために事前にsetにしていた。今は不要
            if isinstance(word_to_pub_set[use_words_sorted[0]], set):
                all_pub_set = word_to_pub_set[use_words_sorted[0]].copy()
            else:
                if not use_words_sorted[0] in pub_set_cache:
                    pub_set_cache[use_words_sorted[0]] = set(word_to_pub_set[use_words_sorted[0]])
                all_pub_set = pub_set_cache[use_words_sorted[0]].copy()
            """
            all_pub_set = word_to_pub_set[use_words_sorted[0]]
            
            use_words_final.append(use_words_sorted[0])
            
            cpc_count = 1 if number_to_word[use_words_sorted[0]][:3] == 'cpc' else 0
            
            
            if not len(all_pub_set) <= r:
                for word in use_words_sorted[1:]:
                    
                    if number_to_word[word][:3] == 'cpc' and cpc_count == 1:
                        continue
                    
                    before_len = len(all_pub_set)
                    """
                    if isinstance(word_to_pub_set[word], set):
                        all_pub_set &= word_to_pub_set[word]
                    else:
                        if not word in pub_set_cache:
                            pub_set_cache[word] = set(word_to_pub_set[word])
                        all_pub_set &= pub_set_cache[word]
                    """
                    

                    # all_pub_set = np.intersect1d(all_pub_set, word_to_pub_set[word], assume_unique = True)
                    all_pub_set = cp.intersect1d(all_pub_set, word_to_pub_set[word], assume_unique = True)
                    
                    
                    after_len = len(all_pub_set)
                    
                    # 集合の要素数が減ったときだけ追加
                    if after_len < before_len:
                        use_words_final.append(word)
                        
                        if number_to_word[word][:3] == 'cpc':
                            cpc_count += 1

                    # cpcのワイルドカードを使わないようにしたので、↓はいらない
                    """
                    else:
                        # cpcでなければ追加
                        if number_to_word[word][:3] != 'cpc':
                            use_words_final.append(word)
                    """
                    # use_words_final.append(word)
                    
                    counts.append(len(all_pub_set))

                    # nn_combだけになったら終了
                    if len(all_pub_set) <= r: 
                        break
                    
                    # 近傍だけになったら終了
                    cnt1 = len(all_pub_set)
                    
                    # cnt2 = len(all_pub_set & nn_set)
                    # cnt2 = len(np.intersect1d(all_pub_set, nn_array, assume_unique = True))
                    
                    if cnt1 <= 50:                 
                        t1 = time.time()
                        cnt2 = len(cp.intersect1d(all_pub_set, nn_array, assume_unique = True))
                        # cnt2 = cp.in1d(all_pub_set ,nn_array, assume_unique = True)
                        # cnt2 = cnt2.sum()

                        t_sum += time.time() - t1

                        if cnt1 == cnt2:
                            break
                    
           
            
            
            use_words = tuple(use_words_final)
            use_words_len_max = max(len(use_words), use_words_len_max)

            if debug_count < -1:
                print(counts)

            debug_count += 1
            if len(all_pub_set) <= 50: # len(all_pub_set) <= 50なしだと、多分OOM + 間に合わない
                
            
                all_pub_set = cp.asnumpy(all_pub_set)

                
                nn_comb_set = set(nn_comb)
                all_pub_set = set(all_pub_set)
                # nn_setで、追加できるなら追加
                add_set = all_pub_set & nn_set
                nn_comb_set = nn_comb_set | add_set
                
                # neg_pub_set = all_pub_set - nn_comb_set

                # if len(neg_pub_set) == 0:
                if len(nn_comb_set) == len(all_pub_set): # negative=0なら追加
                    add_list.append((use_words, nn_comb_set, all_pub_set))

        if i <= 50:
            print(t_sum)
            print(f'r >= {r} len(add_list): {len(add_list)}')        
            print('use_words_len_max ', use_words_len_max )

    add_count = 0
    for word_pattern, pub_set, all_pub_set in add_list:
        if word_pattern in word_pattern_to_pub_set:
            continue

        word_pattern_to_pub_set[word_pattern] = pub_set
        word_pattern_to_pub_set_all[word_pattern] = all_pub_set

        add_count += 1

    if i <= 50:
        print(f'len(add_list): {len(add_list)}, add_count: {add_count}')
    add_count_list.append(add_count)


In [ ]:
print(pd.DataFrame(add_count_list).describe())

In [ ]:
"""
def count_query_len(word_pattern):
    is_replace_AND = False
    prefix_set = set([number_to_word[word].split(':')[0] for word in word_pattern])
    if 'cpc' not in prefix_set:
        is_replace_AND = True
    
    if len(word_pattern) == 1 or is_replace_AND:
        token_len = 2
    else:
        token_len = 3
    return token_len
"""
# cpcもtoken_len=2でできるようになったので、変更
def count_query_len(word_pattern):
    token_len = 2 
    return token_len

In [ ]:
def filtering_word_pattern_to_pub_set_list(word_pattern_to_pub_set_list):
    
    word_pattern_to_pub_set_list_new = []
    # for word_pattern_to_pub_set in word_pattern_to_pub_set_list:
    for sample_idx, word_pattern_to_pub_set in enumerate(word_pattern_to_pub_set_list):

        word_pattern_to_pub_set_new = defaultdict(set)
        sort_keys = []

        # データの処理とソートキーの生成
        for word_pattern, pub_set in word_pattern_to_pub_set.items():
            max_count = len(word_pattern_to_pub_set_all[word_pattern])
            if max_count <= PUB_MAX_COUNT:
                # word_pattern_to_pub_set_new[word_pattern] = pub_set

                neg_pub_set = word_pattern_to_pub_set_all[word_pattern] - pub_set                
                neg_weight = len(neg_pub_set)

                sort_key = (len(pub_set), -count_query_len(word_pattern), -neg_weight)
                sort_keys.append((sort_key, word_pattern))
                word_pattern_to_pub_set_new[word_pattern] = pub_set

        # ソートキーでソート
        sorted_word_patterns = sorted(sort_keys, key=lambda x: x[0], reverse=True)

        # ソート結果に基づいて新しい辞書を作成
        word_pattern_to_pub_set_new = [(word_pattern, word_pattern_to_pub_set_new[word_pattern]) for _, word_pattern in sorted_word_patterns]
        
        # 重複処理 
        seen = set()
        new_state = []
        for word_pattern, pub_set in word_pattern_to_pub_set_new:
            # pattern = tuple(sorted(list(pub_set)))
            pattern = (tuple(sorted(list(pub_set))), count_query_len(word_pattern))
            if pattern in seen:
                continue
            seen.add(pattern)
            new_state.append((word_pattern, pub_set))
        word_pattern_to_pub_set_new = new_state
        
        
        # (検索結果内のtarget数、検索結果内の全てのpublication_number数)でソートした上位 PATTERN_NUM_MAX 件
        word_pattern_to_pub_set_new = word_pattern_to_pub_set_new[:PATTERN_NUM_MAX]
            
        # この処理いらない。new_stateと同じデータ構造なので。
        word_pattern_to_pub_set = []
        for word_pattern, pub_set in word_pattern_to_pub_set_new:
            word_pattern_to_pub_set.append((word_pattern, pub_set))
        
        word_pattern_to_pub_set_list_new.append(word_pattern_to_pub_set)

    word_pattern_to_pub_set_list = word_pattern_to_pub_set_list_new
    
    return word_pattern_to_pub_set_list

In [ ]:
word_pattern_to_pub_set_list = filtering_word_pattern_to_pub_set_list(word_pattern_to_pub_set_list)

In [ ]:
recall_list = []
for word_pattern_to_pub_set in word_pattern_to_pub_set_list:
    true_set = set()
    # for word_pattern, true_pub_set in word_pattern_to_pub_set.items():
    for word_pattern, true_pub_set in word_pattern_to_pub_set:
        true_set |= true_pub_set
    recall_list.append(len(true_set))

print(pd.DataFrame(recall_list).describe())
plt.hist(recall_list)
plt.title('recall distribution')
plt.show()

In [ ]:
# cpcの組み合わせ数
lis = []
for word_pattern_to_pub_set in word_pattern_to_pub_set_list:
    lis.append(len(word_pattern_to_pub_set))
print(pd.DataFrame(lis).describe())
plt.hist(lis)
plt.title('combincation count')

In [ ]:
from functools import total_ordering

@total_ordering
class State:
    def __init__(self, use_word_pattern_list, not_use_word_pattern_list, use_true_pub_set, neg_count, score=None,
                 is_hard_penalty=False):
        self.use_word_pattern_list = use_word_pattern_list
        self.not_use_word_pattern_list = not_use_word_pattern_list
        # self.use_pub_set = use_pub_set
        self.use_true_pub_set = use_true_pub_set
        self.neg_count = neg_count
        
        # 簡単にrecall上げられるサンプルなら、negativeの重みを強める
        self.is_hard_penalty = is_hard_penalty
    
    def __lt__(self, other):
        return self.score < other.score

    def __eq__(self, other):
        return self.score == other.score
    
    def calc_score(self):
        
        # self.score = SUB_MAX_COUNT * (len(self.use_true_pub_set) - 0.25 * self.neg_count)
        
        if self.is_hard_penalty:
            # self.score = len(self.use_true_pub_set) - 0.25 * self.neg_count
            self.score = len(self.use_true_pub_set) - NEG_WEIGHT * self.neg_count
        else:
            self.score = len(self.use_true_pub_set) - NEG_WEIGHT * self.neg_count

In [ ]:
class Timer:
    def __init__(self):
        self.start = time.time()

    def get_current_time(self):
        return (time.time() - self.start)
    
# 焼きなましのための確率を計算
def calc_sa_p(new_score, score, T):
    score_diff = new_score - score
    if score_diff >= 0:
        return 1
    else:
        return math.exp(score_diff / T)

In [ ]:
def ap50(preds, labels):
    precisions = list()
    n_label = len(labels)
    n_found = 0
    for e, i in enumerate(preds):
        if i in labels:
            n_found += 1
        precisions.append(n_found/(e+1)) # this is the line that is probably incorrect for competition 
    return sum(precisions)/50

In [ ]:
len(neighbors.flatten())

In [ ]:
word_pattern_to_neg_list = []

for sample_idx, word_pattern_to_pub_set in tqdm(enumerate(word_pattern_to_pub_set_list)):
    
    word_pattern_to_neg_count = dict()
    for c, pub_set in word_pattern_to_pub_set:
        neg_pub_set = word_pattern_to_pub_set_all[c] - pub_set
        
        word_pattern_to_neg_count[c] = len(neg_pub_set)
    
    word_pattern_to_neg_list.append(word_pattern_to_neg_count)

In [ ]:


def word_pattern_to_query(word_pattern):
    query_rm_cpc = [s for s in word_pattern if number_to_word[s].split(':')[0] != 'cpc']
    query_cpc = [s for s in word_pattern if number_to_word[s].split(':')[0] == 'cpc']
    word_pattern = query_rm_cpc + query_cpc
    
    query = ''
    for i, word in enumerate(word_pattern):
        if number_to_word[word].split(':')[0] == 'cpc':
            if i != len(word_pattern) - 1:
                word = number_to_word[word]

                # cpc:A01H6/77* これだと770~779も含まれるので、この後に対処する
                query += f'{word}*'

                # cpc:A01H6/7? を追加して70~79に制限する。
                base = word.split('/')[0]
                suffix = word.split('/')[1]
                suffix = suffix[:-1] + '?'
                query += f'{base}/{suffix}'
            else:
                # 最後なら、ワイルドカードを追加する必要がない
                query += f'{number_to_word[word]}'
        else:
            query += f'{number_to_word[word]}-'
    
    # 最後のは取り除く
    # if len(word_pattern) >= 2 and query[-1] != '?':
    if query[-1] == '-':
        query = query[:-1]
    
    query = "(" + query + ')'
    return query

In [ ]:
word_pattern_to_char_len = dict()

for word_pattern_to_pub_set in word_pattern_to_pub_set_list:
    for word_pattern, _ in word_pattern_to_pub_set:
        query = word_pattern_to_query(word_pattern)
        word_pattern_to_char_len[word_pattern] = len(query) + 4

In [ ]:
len(word_pattern_to_char_len)

In [ ]:
counts = []
for w, c in word_pattern_to_char_len.items():
    counts.append(c)

print(pd.DataFrame(counts).describe())
plt.hist(counts)

In [ ]:
query_list = []
default_query = 'ti:bread'

# 結果の可視化用
success_count = 0
score_list = []
true_pub_set_len_list = [0] * len(word_pattern_to_pub_set_list)
pub_set_len_list = [0] * len(word_pattern_to_pub_set_list)
neg_count_list = [0] * len(word_pattern_to_pub_set_list)

# for _, word_pattern_counter in tqdm(enumerate(word_pattern_counter_list), total=len(word_pattern_counter_list)):
for sample_idx, word_pattern_to_pub_set in tqdm(enumerate(word_pattern_to_pub_set_list), total=len(word_pattern_to_pub_set_list)):
    word_pattern_to_neg_count = word_pattern_to_neg_list[sample_idx]
    
    true_set = set(nn_df.values[sample_idx, 1:])
    labels = list(nn_df.values[sample_idx, 1:])
    labels = [num_to_pub[pub] for pub in labels]
    
    # 検索クエリの候補が0のとき
    if len(word_pattern_to_pub_set) == 0:
        score_list.append(0)
        query_list.append(default_query)
        continue
    
    word_pattern_to_pub_set_dict = dict()
    for word_pattern, pub_set in word_pattern_to_pub_set:        
        word_pattern_to_pub_set_dict[word_pattern] = pub_set
    
    use_word_pattern_list = []
    not_use_word_pattern_list = []
    for word_pattern, _ in word_pattern_to_pub_set:
        not_use_word_pattern_list.append(word_pattern)
        
    # count_query_len に2秒くらい取られているので、事前計算
    word_pattern_to_query_len = dict()
    for word_pattern, _ in word_pattern_to_pub_set:
        word_pattern_to_query_len[word_pattern] = count_query_len(word_pattern)
        

    curr_state = State([], [], set(), 0, 0)
    curr_state.score = 0
    
    # queryの差分更新用
    curr_query_len = -1
    best_query_len = -1
    curr_char_len = -4
    best_char_len = -4

    
    best_states = []
    timer = Timer()
    
    # 焼きなましによるスコアの変化の可視化用
    _score_list = []
    
    best_state = copy.deepcopy(curr_state)
    # best_state.calc_score()
    best_state.score = 0
    
    # どの近傍が使われたかをcount
    pattern_count = [0, 0]
    
    # debugで時間計測用
    time_sum = [0]
    
    # neg_pickedのペナルティを強めるか
    is_hard_penalty = False
    
    while True:
        curr_time = timer.get_current_time()
       
        # if curr_time > max_time:
        if curr_time > max_time:
            break
            
        t = curr_time / max_time
        T = T0**(1-t) * T1**t
        
        p = random.random()
        
        if p >= 0.5:
            act = 'add_pattern'
        else:
            act = 'remove_pattern'

        next_state = State([], 
                           [], 
                           set(), 0, 0,
                           is_hard_penalty)

        if act == 'add_pattern':
            N = len(not_use_word_pattern_list)
            if N == 0:
                continue
            idx = random.randint(0, N-1)
            c = not_use_word_pattern_list[idx]
            
            # query_len = count_query_len(c)
            query_len = word_pattern_to_query_len[c]
            char_len = word_pattern_to_char_len[c]
        
            if curr_query_len + query_len > 50 or curr_char_len + char_len > CHAR_LIMIT:
                is_exceed_query_limit = True
            else:
                is_exceed_query_limit = False
        elif act == 'remove_pattern':
            N = len(use_word_pattern_list)
            if N == 0:
                continue
            idx = random.randint(0, N-1)
            c = use_word_pattern_list[idx]
            
            query_len = word_pattern_to_query_len[c]
            char_len = word_pattern_to_char_len[c]
            is_exceed_query_limit = False

        if is_exceed_query_limit:
            continue
        
        # 差分更新にして高速化 add_pattern 0.6s remove_pattern 1.4s
        if act == 'add_pattern':
            neg_count = curr_state.neg_count + word_pattern_to_neg_count[c]
            use_true_pub_set = curr_state.use_true_pub_set | word_pattern_to_pub_set_dict[c]
            
            update_count = len(use_true_pub_set) - len(curr_state.use_true_pub_set)
            # recall向上しないならcontinue
            if update_count == 0:
                continue
        else:
            neg_count = 0
            use_true_pub_set = set()

            for word_pattern in use_word_pattern_list:
                if act == 'remove_pattern' and word_pattern == c:
                    continue

                neg_count += word_pattern_to_neg_count[word_pattern]
                if len(use_true_pub_set) + neg_count > SUB_MAX_COUNT:
                    break
                   
                use_true_pub_set |= word_pattern_to_pub_set_dict[word_pattern]
            
        
        # 検索結果数が閾値より大きいときは候補にしない。
        if len(use_true_pub_set) + neg_count > SUB_MAX_COUNT or neg_count > NEG_MAX_COUNT:
            continue

        # next_state.use_pub_set = use_pub_set
        next_state.neg_count = neg_count
        next_state.use_true_pub_set = use_true_pub_set
        next_state.calc_score()
        
        # 新しい状態を受理する確率
        sa_p = calc_sa_p(next_state.score, curr_state.score, T)

        if random.random() < sa_p:
            curr_state = next_state
            # curr_query_list = next_query_list
            if act == 'add_pattern':
                c = not_use_word_pattern_list.pop(idx)
                use_word_pattern_list.append(c)
                curr_query_len += query_len
                curr_char_len += char_len
            elif act == 'remove_pattern':
                c = use_word_pattern_list.pop(idx)
                not_use_word_pattern_list.append(c)
                curr_query_len -= query_len
                curr_char_len -= char_len
        else:
            pass

        # _score_list.append(len(curr_state.use_true_pub_set))
        _score_list.append(curr_state.score)
        
        
        if curr_state.score > best_state.score:
            best_state = curr_state
            best_state.use_word_pattern_list = copy.deepcopy(use_word_pattern_list)
            # best_query_list = copy.deepcopy(curr_query_list)
            best_query_len = curr_query_len
            best_char_len = curr_char_len
            
            if best_state.score >= 35 and not is_hard_penalty:
                is_hard_penalty = True
                curr_state.is_hard_penalty = True
                curr_state.calc_score()
                best_state.is_hard_penalty = True
                best_state.calc_score()
        
        
        if act == 'add_pattern':
            pattern_count[0] += 1
        else:
            pattern_count[1] += 1
    # print(time_sum)
    # print(pattern_count)
    # debug用
    if sample_idx < 50:
        plt.plot(_score_list)
        plt.show()
    
    # best_stateで提出用のqueryを作る
    if best_state is None:
        print('error: best_state is None')
        print(1/0)
        # query_list.append(default_query)
    else:
        """
        use_true_pub_set = best_state.use_true_pub_set
        use_pub_set = best_state.use_pub_set
        """
        use_true_pub_set = set()
        use_pub_set = set()
        for c in best_state.use_word_pattern_list:
            use_pub_set |= word_pattern_to_pub_set_all[c]
            use_true_pub_set |= word_pattern_to_pub_set_dict[c]
        
        use_word_pattern_list = best_state.use_word_pattern_list
        
        true_pub_set_len_list[sample_idx] = len(use_true_pub_set)
        pub_set_len_list[sample_idx] = len(use_pub_set)
        neg_count_list[sample_idx] = best_state.neg_count
        
        query = ''
        for i, word_pattern in enumerate(use_word_pattern_list):
            prev_query = query

            _query = word_pattern_to_query(word_pattern)
            
            query += _query
            
            if i != len(use_word_pattern_list) - 1:
                query += ' OR '
                
            if whoosh_utils.count_query_tokens(query) > 50 or len(query) > CHAR_LIMIT + 500:
 
                # エラー出して止まるようにする
                print('query limit error')
                print(1/0)
                
                query = prev_query
                break
        
        # assert best_query_len == len(query.split())
            
        query_list.append(query)
        success_count += 1
        
        if IS_TRAIN:
            results = whoosh_utils.execute_query(query, qp, searcher)
            result_set = set(results)
            n_pick = len(set(labels) & result_set)
            score = ap50(results + [-1] * (50 - len(results)), labels)
        else:
            score = 0
            n_pick = 0
        
        if sample_idx < 50:
            print(query)
            print('ap50', score)
            print('n_pick', n_pick)
            print('true_count', true_pub_set_len_list[sample_idx])
            print('neg_count', neg_count_list[sample_idx])
            print(len(query), best_char_len)
        
        score_list.append(score)

In [ ]:
np.mean(true_pub_set_len_list)

In [ ]:
plt.hist(true_pub_set_len_list)
plt.title('true_pub_set_len_list distribution')

In [ ]:
np.mean(score_list)

In [ ]:
plt.hist(score_list)
plt.title('score_list distribution')

In [ ]:
np.mean(pub_set_len_list)

In [ ]:
plt.hist(pub_set_len_list)
plt.title('pub_set_len_list distribution')

In [ ]:
np.mean(neg_count_list)

In [ ]:
plt.hist(neg_count_list)
plt.title('neg_count_list distribution')

In [ ]:
neg_count_list_2 = [pub_set_len_list[i] - true_pub_set_len_list[i]
                  for i in range(len(pub_set_len_list))]
np.mean(neg_count_list_2)

In [ ]:
plt.hist(neg_count_list_2)
plt.title('neg_count_list_2 distribution')

In [ ]:
neg_count_list_diff = [neg_count_list[i] - neg_count_list_2[i] for i in range(len(neg_count_list))]
print(np.mean(neg_count_list_diff ))
plt.hist(neg_count_list_diff)
plt.title('neg_count_list_diff distribution')

In [ ]:
if IS_TRAIN:
    [print(s) for s in true_pub_set_len_list[:50]]
else:
    print(score_list[:10])

In [ ]:
if IS_TRAIN:
    [print(s) for s in score_list[:50]]
else:
    print(score_list[:10])

In [ ]:
query_count_list = [whoosh_utils.count_query_tokens(query) for query in query_list]
print(max(query_count_list))
plt.hist(query_count_list)

plt.title('token len distribution')
plt.show()

In [ ]:
character_len_list = [len(query) for query in query_list]
print(max(character_len_list))
plt.hist(character_len_list)
plt.title('character len distribution')
plt.show()

In [ ]:
len(nn_df), len(query_list), success_count

# make oof

In [ ]:
if IS_TRAIN:
    columns = ['query_list',
               'true_pub_set_len_list',
               'score_list',
               'pub_set_len_list',
               'neg_count_list',
               'neg_count_list_2',
               'query_count_list',
               'character_len_list']
    values = np.array([ query_list,
               true_pub_set_len_list,
               score_list,
               pub_set_len_list,
               neg_count_list,
               neg_count_list_2,
               query_count_list,
               character_len_list]).T
    
    
    oof_df = pd.DataFrame(values, columns=columns)
    
    print(oof_df.head())
    oof_df.to_csv('oof_df.csv', index=False)

In [ ]:
plt.scatter(true_pub_set_len_list, score_list)
plt.xlabel('true_pub_set_len_list')
plt.ylabel('score_list')

In [ ]:
idxs = [i for i in range(len(true_pub_set_len_list)) if score_list[i] != 0]
true_pub_set_len_list = [true_pub_set_len_list[i] for i in idxs]
score_list = [score_list[i] for i in idxs]
print(len(true_pub_set_len_list))
plt.scatter(true_pub_set_len_list, score_list)
plt.xlabel('true_pub_set_len_list')
plt.ylabel('score_list')

# submission

In [ ]:
default_query = 'ti:bread'

for i in range(len(query_list)):
    if 'id:' in query_list[i]:
        query_list[i] = default_query

"""
for i in range(len(query_list)):
    try:
        result = whoosh_utils.execute_query(query_list[i], qp, searcher)
    except Exception as e:
        query_list[i] = default_query
"""

In [ ]:
for i in range(len(query_list)):
    if query_list[i] == '':
        query_list[i] = default_query

In [ ]:
"""
if not IS_TRAIN:
    # train_idx = whoosh_utils.load_index('/kaggle/input/create-valid-index-add-claim/test_index')
    train_idx = whoosh_utils.load_index('/kaggle/input/uspto-test-index/test_index')
    searcher = whoosh_utils.get_searcher(train_idx)
    qp = whoosh_utils.get_query_parser()

    # query = 'ti:balloons OR ti:string'
    query = '(cpc:A01H6/42 ab:cuttings) OR (ab:regal-ab:rooting) OR (ab:magenta-ab:umbels) OR (ti:geranium-ti:variety-ab:cherry-ab:foliage-ab:pink-clm:geranium) OR (ti:bravo-ab:bicolored) OR (ab:foliage-ab:geranium-ab:regal-clm:geranium-detd:duchess) OR (ti:debutante-ti:pelargonium) OR (ti:geranium-ab:bright-ab:garden-ab:geranium-ab:mounded-ab:zonal-clm:geranium-clm:oglger14007-clm:zonal-detd:oglger14007) OR (ti:geranium-ti:variety-ab:blossoms-ab:foliage-ab:orchid-clm:geranium) OR (ti:louise-ab:ivy) OR (ti:hots) OR (ti:geranium-ti:variety-ab:orange-ab:rain-clm:geranium)'
    print(whoosh_utils.execute_query(query, qp, searcher)[:5])
"""

In [ ]:
"""
for i in range(len(query_list)):
    try:
        result = whoosh_utils.execute_query(query_list[i], qp, searcher)
    except Exception as e:
        print(e)
        query_list[i] = default_query
"""

In [ ]:
if not IS_TRAIN:
    sub = pd.read_csv('/kaggle/input/uspto-explainable-ai/sample_submission.csv')
    
    sub['query'] = query_list
    
    sub.to_csv('submission.csv', index=False)
    
    print(sub)